<a href="https://colab.research.google.com/github/Hamza-333/Text-Summarizer/blob/main/413_Final_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.3 MB/s eta 0:00:00


In [3]:
from datasets import list_datasets, load_dataset

In [4]:
datasets = list_datasets()

In [5]:
# load cnn-daily mail
# cnn = load_dataset('cnn_dailymail', '1.0.0')

In [6]:
# cnn['train'][0]

In [7]:
# load multi news
# multi_news = load_dataset('multi_news')

In [8]:
# # load stacked sum
# stacked_sum = load_dataset('stacked-summaries/stacked-xsum')

In [9]:
# using around 100k train, 19k validation, 1.95k test
gigaword = load_dataset('gigaword', split = ['train[:3%]', 'validation[:10%]', 'test'])

Generating train split:   0%|          | 0/3803957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/189651 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1951 [00:00<?, ? examples/s]

Dataset gigaword downloaded and prepared to /root/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# rename columns to have same column names across datasets
# cnn = cnn.rename_column("highlights", 'summary')
# cnn = cnn.rename_column("article", 'document')

In [11]:
# delete irrelevant columns
# cnn = cnn.remove_columns("id")
# stacked_sum = stacked_sum.remove_columns(['id','chapter_length', 'summary_length',\
#                             'is_stacked'])

In [12]:
from datasets.combine import concatenate_datasets

In [13]:
# concatenate all datasets
# train_data = concatenate_datasets([cnn['train'], multi_news['train']
#                                   ])
# val_data = concatenate_datasets([cnn['validation'], multi_news['validation']])
# test_data = concatenate_datasets([cnn['test'], multi_news['test']])

In [14]:
# train_data = cnn['train']
# val_data = cnn['validation']
# test_data = cnn['test']

# only using gigaword for now
train_data = gigaword[0]
val_data = gigaword[1]
test_data = gigaword[2]

In [15]:
# save data so we don't have to redownload every time

# train_data.to_csv('train.csv')
# val_data.to_csv('val.csv')
# test_data.to_csv('test.csv')

In [16]:
train_data[0]

{'document': "australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .",
 'summary': 'australian current account deficit narrows sharply'}

# Data Preparation

In [17]:
import torch

In [18]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.2 MB/s eta 0:00:00


In [19]:
from transformers import AutoTokenizer

In [20]:
train_data[0]['document']

"australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed ."

In [21]:
bos = '<|bos|>'
eos = '<|eos|>'
pad = '<|pad|>'
special_tokens_dict = {'bos_token': bos, 'pad_token': pad, 'eos_token':eos}

In [22]:
from transformers import GPT2TokenizerFast

In [23]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', add_bos_token = True) #transformer library again but now with post processing
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [24]:
def preprocess_function(examples):
    """Create tokens for each example. Padding is done using the max length of
    example in the batch. Each example in the batch will have equal length"""
    # inputs = [doc for doc in examples['document']]
    # fixed size padding, hard coded for now
    model_inputs = tokenizer(examples['document'], padding='max_length', \
                             max_length=75)

    labels = tokenizer(text_target=examples["summary"], padding='max_length', \
                       max_length=52)

    # print(labels)
    model_inputs["labels"] = labels["input_ids"]

    model_inputs["labels_mask"] = labels["attention_mask"]

    # store as pairs of documents and summaries
    # store the vocabulary indexes of each word
    # inp_lst.append((model_inputs["input_ids"], labels["input_ids"], \
                    # model_inputs['attention_mask'], labels['attention_mask']))

    return model_inputs

In [25]:
import torch.nn as nn

In [26]:
from transformers import DataCollatorWithPadding

In [27]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [60]:
# preprocessing function tokenizes in batches
train_data = train_data.map(preprocess_function, batched = True)

Map:   0%|          | 0/114119 [00:00<?, ? examples/s]

In [61]:
# only keep the encoded columns
train_d = train_data.remove_columns(['document', 'summary'])

In [62]:
# format to pytorch
train_d = train_d.with_format("torch")

In [63]:
vocab_size = len(tokenizer.vocab)

In [32]:
EMB_DIM = 50

In [33]:
import math

In [34]:
class PositionalEncoding(nn.Module):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def positional_encoding(self, dim_model, length):
        pos_encod = torch.zeros(length, dim_model)
        pos_list = torch.arange(0, length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) #1000^(2i/dim_model)

        pos_encod[:, 0::2] = torch.sin(pos_list * div_term) # pe(even) = sin(pos/1000^(2i/dim_model))
        pos_encod[:, 1::2] = torch.cos(pos_list * div_term) # pe(odd) = cos(pos/1000^(2i/dim_model))
        if torch.cuda.is_available():
            pos_encod= pos_encod.cuda()
        return pos_encod
    
    def forward(self, embedding, dim_model, length):
        return embedding + self.positional_encoding(dim_model, length)


In [44]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=EMB_DIM, nhead=25, num_encoder_layers=16, \
                 num_decoder_layers=12, dim_feedforward=4096, dropout=0):
        super(TransformerModel, self).__init__()

        self.pos = PositionalEncoding()
        # transformer model with encoder and decoders
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers,
                dim_feedforward, dropout, activation='relu', layer_norm_eps=1e-06, batch_first=True)
        if torch.cuda.is_available():
            self.transformer.cuda()
        # final fully connected linear layer for classification
        self.fc = nn.Linear(d_model, vocab_size)



    def forward(self, inp, tgt, src_pad_mask=None, tgt_pad_mask=None):

        # Positional encoding for input and target

        inp = self.pos(inp, EMB_DIM, inp.size(1))
        tgt = self.pos(tgt, EMB_DIM, tgt.size(1))
        # target mask for decoder
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1), device='cuda')
        # encoder and decoder of transformer
        out = self.transformer(inp, tgt, tgt_mask=tgt_mask, \
                               src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)

        # final linear layer
        out = self.fc(out)

        return out

    

In [36]:
import torch.optim as optim

In [37]:
import random

In [38]:
from torchtext.vocab import GloVe
# smallest glove embedding vectors
glove = GloVe(name='6B', dim=50)
# can also try 300d vectors (take much longer to train and need more gpu/ram)
# if using 300d vectors --> set EMB_DIM = 300
# glove = GloVe(name='42B', dim=300)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:10<00:00, 39932.67it/s]


In [ ]:
import pickle

In [39]:

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# save tokens
# with open("/content/gdrive/My Drive/inputs.pkl", "wb") as fp:   
#     pickle.dump(inp_lst, fp)

In [ ]:
# Only run if you have saved the file
with open("/content/gdrive/My Drive/inputs.pkl", "rb") as fp: 
    inp_lst = pickle.load(fp)

In [40]:
def glove_embed(data):
    """Get glove embeddings of sequences"""
    max_length = 0 
    # find the maximum length of a sequence in the batch

    for i in range(len(data)):
        max_length = max(max_length, len(data[i]))
    # create empty tensor of size len(data), max_length, size of glove embedding

    tensor = torch.empty(len(data), max_length, EMB_DIM)
    for i in range(len(data)):
        # get glove embedding, use lower case back up if token not found
        words = tokenizer.convert_ids_to_tokens(data[i])
        emb = glove.get_vecs_by_tokens(words, lower_case_backup=True)
        tensor[i] = emb

    return tensor

In [ ]:
import gc

In [41]:
def get_summary(output):
    """Converts model output to tokens"""
    lst = []

    # for i in range(output.size(0)):
    pred = output.max(1, keepdim=True)[1]

    # break
    words = tokenizer.convert_ids_to_tokens(pred)
    w = tokenizer.decode(pred[0])

    print(words)

In [ ]:
# def get_accuracy(model, inp, tgt):
#     # note: why should we use a larger batch size here?
#     # We do not need to compute paramater updates, thus a smaller batch size is 
#     # not advantageous. A larger batch size will reduce number of iterations
#     # and would allow for more parallel computations. Therefore, it would be 
#     # faster.


#     model.eval() # annotate model for evaluation (why do we need to do this?)
#     # some layers act differently in training vs testing, e.g: BatchNorm, 
#     # Dropout. Need to "turn them off" when calculating accuracy.
#     correct = 0
#     total = 0
#     N = 0
#     # placeholder end of text token
#     y_input = tokenizer.convert_ids_to_tokens(50256)
#     y_input = glove.get_vecs_by_tokens(y_input)
#     if torch.cuda.is_available():
#         model.cuda()
#     for i, t in zip(inp, tgt):
#         if torch.cuda.is_available():
#             i = i.cuda()
#             t = t.cuda()
#         output = model(i, y_input)               
#         pred = output.max(1, keepdim=True)[1] 
#         correct += pred.eq(tgt.view_as(pred)).sum().item()
#         N += inp.shape[0]
    
#     return correct/N # TODO: return the model accuracy

In [79]:
def train(model, train_data, batch_size=64, weight_decay=0.0,
           learning_rate=0.000008, num_epochs=30, checkpoint_path=None):

    model.train()
    # ignore padding tokens in loss computation
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    # load data with batch size of 64
    loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, collate_fn=data_collator,\
                                         shuffle=True)

    epoch = 0
    train_accs, train_losses = [], []
    val_accs = []
    iters = []
    n = 0
    epochs = []
    t_loss = 0
    if torch.cuda.is_available():
        model.cuda()
        criterion.cuda()
 
    batch = 0
    while epoch < num_epochs:

        learning_rate = learning_rate
        # Using AdamW for now, can try with other optimizers too
        optimizer = optim.Adam(model.parameters(),
                lr=learning_rate,
                weight_decay=weight_decay)
        
        i = 1
        t_loss = 0
        for step, batch in enumerate(loader):
            # input token ids
            inp = batch['input_ids']
            # input target ids
            tgt_ind = batch['labels']
            # input padding mask
            inp_mask = batch['attention_mask']
            # target padding mask
            tgt_mask = batch['labels_mask']

            if len(inp) < batch_size:
                # Last batch will have less images since it will be the
                # remainder of Total images / batch_size.

                # Adjust the learning rate of the last batch by 
                # (size(last_batch) / batch_size) to account 
                # for the smaller size.
                learning_rate = learning_rate * (len(inp) / batch_size)
                optimizer = optim.Adam(model.parameters(),
                    lr=learning_rate,
                    weight_decay=weight_decay)

            if torch.cuda.is_available():
                inp = inp.cuda()
                tgt_ind = tgt_ind.cuda()

            device = "cuda:0" if torch.cuda.is_available() else "cpu"
            # padding mask so that model ignores padding tokens
            # matrix of booleans that tells model where padding tokens are
            # so that it can ignore
            inp_mask = (inp_mask > 0).type(torch.float32).to(device)
            tgt_mask = (tgt_mask > 0).type(torch.float32).to(device)
            # shift target output to the right 

            tgt_shifted = torch.zeros(tgt_ind.size())
            tgt_shifted[:, 1:] = tgt_ind[:, :-1].clone()
            tgt_shifted[:, 0] = tokenizer.bos_token_id
            # print(tgt_shifted.size())
            # print(tgt_ind.size())
            # get glove embeddings
            inp = glove_embed(inp).to(device)
            tgt = glove_embed(tgt_shifted).to(device)
            # with torch.autocast(device_type='cuda', dtype=torch.float16):


            outputs = model(inp, tgt, inp_mask, tgt_mask)

            outputs = torch.permute(outputs, (0,2,1))

            # normalize loss
            # compare output against the target indices (not the glove embedded target)
            loss = criterion(outputs, tgt_ind) / batch_size

            loss.backward()
            n += 1

            
            # update parameters every fourth iteration so we can accumalte gradients
            # over smaller batches --> conserves memory

            if i % 4 == 0 or (i == len(train_data)):
                # print(i)
                optimizer.step()

                optimizer.zero_grad()
                t_loss += float(loss)

            i += 1

                
            if i % 100 == 0:
                # get_summary prints out model's output tokens
                # this is just to see what the model is outputting for one sample
                get_summary(torch.permute(outputs, (0,2,1))[0])
                print(i)
            
            iters.append(n)


        # end of epoch
        # break

        epoch += 1
        torch.save(model.state_dict(), checkpoint_path.format(epoch))
        print("Train loss: {} -----------------  epoch: {}".format(t_loss, epoch))
        

In [ ]:
# vocab_size=50257

In [80]:
model = TransformerModel(vocab_size=vocab_size)

In [48]:
# load partially trained to train further
model = TransformerModel(vocab_size=vocab_size)
model.load_state_dict(torch.load\
                      ('/content/gdrive/My Drive/Transformerckpt-5.pk'))

<All keys matched successfully>

In [ ]:
train(model, train_d, checkpoint_path='/content/gdrive/My Drive/CSC413/A2/Transformer2ckpt-{}.pk')

['ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic', 'ublic']
100
['group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group', 'group']
200
['Ġhubs', 'Ġhubs', 'Ġhubs', 'Ġhubs', 'Ġhubs', 'Ġhubs',

In [ ]:
model = TransformerModel(vocab_size=vocab_size)
model.load_state_dict(torch.load\
                      ('/content/gdrive/My Drive/Transformerckpt-5.pk'))

<All keys matched successfully>

In [ ]:
inp_lst[3][0][0]

tensor([28984,   452,   666,  1893,   819,    78,  2146,  2040,   319, 29445,
         3462,  6875,   262,   514, 14791,   284,  8591,   279,  1031,  7559,
        27822,  1729,  1036,  1045,   837, 10148, 19786,   262, 39137,   286,
        12577,   262, 41584,   286,   262,  1499,   416, 11560, 22439,  1042,
          764, 50256, 50256, 50256, 50256, 50256, 50256])

In [ ]:
[[50256]]*16

In [ ]:
# check output of the model
model.eval()
g1 = glove_embed(inp_lst[3][0]).to(device)
g2 = glove_embed([[50256]]*16).to(device)
model.to(device)
summary = model(g1, g2)

In [ ]:
tokenizer.convert_ids_to_tokens(inp_lst[4][0][0])

In [ ]:
summary.size()

torch.Size([16, 1, 50257])

In [ ]:
get_summary(summary[8])

['Ġto']


In [ ]:
tokenizer.decode(summary)

In [ ]:
test_data = test_data.map(preprocess_function, batched = True)

In [ ]:
test_data[:5]['input_ids']

**** DONT RUN ANYTHING UNDER THIS --------------------------------------------------------------** **